In [1]:
import weaviate

weaviate_client = weaviate.connect_to_local()

knowledge_base = weaviate_client.collections.get("KnowledgeBase")

In [12]:
from weaviate.classes.query import Filter

def get_search_results(query: str, source: str) -> str:
    """Sends a query to Weaviate's Hybrid Search. Parases the response into a {k}:{v} string."""
    
    response = knowledge_base.query.hybrid(
        query,
        filters=Filter.by_property("_source").equal(source),
        limit=3
    )
    
    stringified_response = ""
    for idx, o in enumerate(response.objects):
        stringified_response += f"\033[92mSearch Result: {idx+1}\033[0m\n"
        for prop in o.properties:
            stringified_response += f"{prop}: {o.properties[prop]}"
        stringified_response += "\n\n"
    
    return stringified_response

In [13]:
print(get_search_results("What is a DSPy Signature?", source="Docs"))

Search Result: 1
content: When we assign tasks to LMs in DSPy, we specify the behavior we need as a Signature.

A signature is a declarative specification of input/output behavior of a DSPy module. Signatures allow you to tell the LM what it needs to do, rather than specify how we should ask the LM to do it.

You're probably familiar with function signatures, which specify the input and output arguments and their types. DSPy signatures are similar, but the differences are that:

- While typical function signatures just describe things, DSPy Signatures define and control the behavior of modules.

- The field names matter in DSPy Signatures. You express semantic roles in plain English: a question is different from an answer, a sql_query is different from python_code._source: Docssummary: Introduction to DSPy Signatures, which specify the input/output behavior of a DSPy module, allowing you to tell the LM what it needs to do. Unlike typical function signatures, DSPy Signatures define and 

In [16]:
import ollama
from typing import List, Dict

tools_schema=[{
    'type': 'function',
    'function': {
        'name': 'get_search_results',
        'description': 'Get search results for a provided query.',
        'parameters': {
          'type': 'object',
          'properties': {
            'query': {
                'type': 'string',
                'description': 'The search query.',
            },
            'source': {
                'type': 'string',
                'description': 'The source to search through, either "DOCS" or "CODE"' 
            },
          }
        },
        'required': ['query'],
    },
}]

tool_mapping = {
    "get_search_results": get_search_results
}

def ollama_generation_with_tools(user_message: str,
                                 tools_schema: List, tool_mapping: Dict,
                                 model_name: str = "llama3.1") -> str:
    messages=[{
        "role": "user",
        "content": user_message
    }]
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=tools_schema
    )
    if not response["message"].get("tool_calls"):
        return response["message"]["content"]
    else:
        for tool in response["message"]["tool_calls"]:
            function_to_call = tool_mapping[tool["function"]["name"]]
            print(f"Calling function {function_to_call}...")
            query = tool["function"]["arguments"]["query"]
            source = tool["function"]["arguments"]["source"]
            print(f"With args\n\tquery={query}\n\tsource={source}")
            function_response = function_to_call(
                query=query,
                source=source
            )
            messages.append({
                "role": "tool",
                "content": function_response,
            })
    
    final_response = ollama.chat(model=model_name, messages=messages)
    return final_response["message"]["content"]

In [18]:
ollama_generation_with_tools("What is the WeaviateRM?",
                            tools_schema=tools_schema, tool_mapping=tool_mapping)

Calling function <function get_search_results at 0x120f0dcf0>...
With args
	query=WeaviateRM
	source=DOCS


'The WeaviateRM (Retriever Module) is a component of the Weaviate library that enables similarity-based retrieval of data points from an in-memory vector database. It uses the Faiss (Facebook AI Similarity Search) algorithm to efficiently search for similar vectors.\n\nIn the context of the provided code snippets, WeaviateRM is used as part of the DSPy (Data Science Python) framework to create a retriever that can be used with various sentence embedding models, such as those provided by SentenceTransformersVectorizer. This allows for efficient retrieval of similar sentences based on their embeddings.\n\nTo use WeaviateRM, you would typically:\n\n1. Import the necessary libraries and modules.\n2. Create an instance of the WeaviateRM class, passing in a list of document chunks (sentences) to be indexed.\n3. Configure the settings to specify the retriever module (in this case, WeaviateRM).\n4. Perform search queries by calling methods on the WeaviateRM instance.\n\nNote that WeaviateRM is